# O2S model



In [1]:
%load_ext lab_black

import tensorflow as tf
import numpy as np
import pandas as pd

### Parameters block for Papermill
- Instead of using model_cfg directly, this extra step is needed for batch run using Papermill

In [2]:
code_name = 'O2S_v0001'

x_name = 'x_train_4721'
y_name = 'tasa_4721'

sample_name = 'hal'  #hal log frequency with clipping like HS04
sample_rng_seed = 1234
tf_rng_seed = 4321

# Model architechture
o_input_dim = 119
hidden_units = 100

sem_units = 300 if y_name == 'tasa_4721' else 768
cleanup_units = 10
rnn_activation = 'sigmoid'
regularizer_const = 0.

embed_attractor_cfg = None
embed_attractor_h5 = None

p_noise = 0.  # i.e. w_pp, w_pc, and w_cp noise
tau = 1.
max_unit_time = 2.

# Training
n_mil_sample = 1.
batch_size = 128
learning_rate = 0.0001
save_freq = 5

# Results push to BQ database
bq_dataset = None

### Packing parameters into model_cfg

In [3]:
from meta import model_cfg

cfg = model_cfg(
    code_name=code_name,
    x_name=x_name,
    y_name=y_name,
    sample_name=sample_name,
    sample_rng_seed=sample_rng_seed,
    tf_rng_seed=tf_rng_seed,
    use_semantic=False,
    sem_param_gf=0,
    sem_param_gi=0,
    sem_param_kf=0,
    sem_param_ki=0,
    sem_param_hf=0,
    sem_param_hi=0,
    o_input_dim=o_input_dim,
    hidden_units=hidden_units,
    pho_units=sem_units,  # Output become semantic embedding vector
    cleanup_units=cleanup_units,
    embed_attractor_cfg=embed_attractor_cfg,
    embed_attractor_h5=embed_attractor_h5,
    w_oh_noise=0.,
    w_hp_noise=0.,
    w_pp_noise=p_noise,
    w_pc_noise=p_noise,
    w_cp_noise=p_noise,
    tau=tau,
    max_unit_time=max_unit_time,
    n_mil_sample=n_mil_sample,
    batch_size=batch_size,
    rnn_activation=rnn_activation,
    regularizer_const=regularizer_const,
    learning_rate=learning_rate,
    save_freq=save_freq,
    bq_dataset=None
)

# TF random seed (Sampling is out of TF scope... change sample_rng_seed instead)
tf.random.set_seed(cfg.tf_rng_seed)

# Preload data
from data_wrangling import sample_generator, my_data
data = my_data(cfg)

==========Orthographic representation==========
x_train shape: (4721, 119)
x_strain shape: (160, 119)

==========Phonological representation==========
38  phonemes:  dict_keys(['p', 'f', 'm', 'C', 'E', '@', 'e', 'b', 'A', '_', 'S', 'Z', 'z', 's', 'v', 'O', 'n', 'U', 'r', 'y', 'T', 'o', 'u', 'k', '^', 'Y', 'a', 'h', 'w', 'i', 'W', 'l', 'D', 't', 'g', 'd', 'I', 'J'])
y_train shape: (4721, 300)
y_strain shape: (160, 300)


# Modeling

## Building

In [26]:
def build_model(training=True):
    # Organization principal:
    # Structure things, such as repeat vector should build within the model
    # Static calculation of input --> Easier to modify --> build within sample generator

    from tensorflow.keras import Model
    from tensorflow.keras.layers import Layer, Input, concatenate, multiply, RepeatVector, Dense
    from tensorflow.keras.optimizers import Adam
    from modeling import rnn
    #     from modeling_without_cleanup import rnn_no_cleanup_no_pp

    # Train/test mode checking
    cfg.noise_on() if training is True else cfg.noise_off()

    input_o = Input(shape=(cfg.o_input_dim, ), name="Input_O")
    hidden = Dense(cfg.hidden_units)(input_o)
    output = Dense(sem_units)(hidden)
    model = Model(input_o, output)

    model.compile(
        loss='mse',
        optimizer=Adam(
            learning_rate=cfg.learning_rate,
            beta_1=0.9,
            beta_2=0.999,
            amsgrad=False
        ),
        metrics=['accuracy', 'mse']
    )

    model.summary()
    return model


model = build_model(training=True)

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_O (InputLayer)         [(None, 119)]             0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               12000     
_________________________________________________________________
dense_3 (Dense)              (None, 300)               30300     
Total params: 42,300
Trainable params: 42,300
Non-trainable params: 0
_________________________________________________________________


## Arming attractor

In [5]:
if cfg.embed_attractor_cfg is not None:
    print('Found attractor info in config (cfg), arming attractor...')
    from modeling import attractor, arm_attractor
    from evaluate import plot_variables

    attractor_cfg = model_cfg(None)
    attractor_cfg.load_cfg_json(cfg.embed_attractor_cfg)
    attractor_obj = attractor(attractor_cfg, cfg.embed_attractor_h5)

    model = arm_attractor(model, attractor_obj)
    plot_variables(model)
else:
    print('Config indicates no attractor, I have do nothing.')

Config indicates no attractor, I have do nothing.


## Training

In [17]:
df_strain = data.df_strain
df_strain['id'] = df_strain.index
df_strain

,word,ort,pho,wf,frequency,pho_consistency,imageability,img,id
0,ball,__ba_ll___,__bal_____,1393,HF,INC,HI,6.6,0
1,bank,__ba_nk___,__b@nk____,53170,HF,CON,HI,6.1,1
2,beach,__beach___,__biC_____,1691,HF,CON,HI,6.5,2
3,beak,__beak____,__bik_____,10,LF,INC,HI,5.2,3
4,beard,__beard___,__bird____,210,LF,INC,HI,6.3,4
...,...,...,...,...,...,...,...,...,...
155,wool,__wool____,__wUl_____,153,LF,INC,HI,5.8,155
156,worm,__wo_rm___,__w^rm____,100,LF,INC,HI,6.6,156
157,worth,__wo_rth__,__w^rT____,5251,HF,INC,LI,2.3,157
158,wrong,_wro_ng___,__rang____,3195,HF,CON,LI,2.4,158


In [22]:
hfid = df_strain.loc[df_strain.frequency == 'HF', 'id']
lfid = df_strain.loc[df_strain.frequency == 'LF', 'id']

In [27]:
import h5py, pickle, os
from tensorflow.keras.callbacks import ModelCheckpoint
from data_wrangling import sample_generator
from IPython.display import clear_output

checkpoint = ModelCheckpoint(
    cfg.path_weights_checkpoint,
    verbose=1,
    save_freq=cfg.save_freq_sample,
    save_weights_only=True
)

history = model.fit(
    sample_generator(cfg, data),
    validation_data=(data.x_strain, data.y_strain),
    steps_per_epoch=cfg.steps_per_epoch,
    epochs=cfg.nEpo,
    verbose=0,
    callbacks=[checkpoint],
)

# Saving history and model
pickle_out = open(cfg.path_history_pickle, "wb")
pickle.dump(history.history, pickle_out)
pickle_out.close()

clear_output()
print('Training done')

ValueError: Unable to match target structure and sample_weight_modes structure:
  ['...', '...']
    to  
  ['...']

# Reporting

### Training history

In [7]:
from evaluate import training_history

hist = training_history(cfg.path_history_pickle)
hist.plot_all(cfg.path_plot_folder + 'history.html')

alt.HConcatChart(...)

In [8]:
hist.history

,loss,rnn_1_accuracy,rnn_1_loss,rnn_1_mse,rnn_accuracy,rnn_loss,rnn_mse,epoch
0,0.309128,0.004307,0.164160,0.164160,0.002905,0.144968,0.144968,0
1,0.128990,0.005208,0.088960,0.088960,0.003305,0.040030,0.040030,1
2,0.081926,0.006611,0.064021,0.064021,0.002905,0.017905,0.017905,2
3,0.056236,0.005008,0.045946,0.045946,0.001803,0.010290,0.010290,3
4,0.039366,0.005809,0.032720,0.032720,0.001202,0.006646,0.006646,4
...,...,...,...,...,...,...,...,...
95,0.000201,0.005709,0.000106,0.000106,0.011218,0.000096,0.000096,95
96,0.000203,0.005008,0.000107,0.000107,0.010917,0.000097,0.000097,96
97,0.000205,0.005308,0.000107,0.000107,0.009615,0.000098,0.000098,97
98,0.000199,0.006911,0.000104,0.000104,0.012921,0.000095,0.000095,98


### Parse item level stats

In [ ]:
# Must turn training mode off before evaluation
model = build_model(training=False)
from evaluate import strain_eval

# Strain full model
strain = strain_eval(cfg, data, model)
strain.start_evaluate(
    test_use_semantic=True,
    output=cfg.path_model_folder + 'result_strain_item.csv'
)

In [ ]:
# Semantic lesion in Strain
strain_ns = strain_eval(cfg, data, model)
strain_ns.start_evaluate(
    test_use_semantic=False,
    output=cfg.path_model_folder + 'result_strain_ns_item.csv'
)

In [ ]:
# Grain
model = build_model(training=False)
from evaluate import strain_eval, grain_eval
grain = grain_eval(cfg, data, model)
grain.start_evaluate(
    test_use_semantic=False,
    output=cfg.path_model_folder + 'result_grain_item.csv'
)

### Strain plots

In [ ]:
from evaluate import vis

vis_ns = vis(
    cfg.path_model_folder, 'result_strain_ns_item.csv', 'result_grain_item.csv'
)

vis = vis(
    cfg.path_model_folder, 'result_strain_item.csv', 'result_grain_item.csv'
)

vis_ns.parse_cond_df()
vis.parse_cond_df()

full = vis.plot_dev('acc').properties(title='Full input')
lesion = vis_ns.plot_dev('acc').properties(title='Semantic lesion')

strain_plot = full | lesion
strain_plot.save(cfg.path_plot_folder + 'strain.html')
strain_plot

### Lesion development deep dive

In [ ]:
dev_inter = vis_ns.plot_dev_interactive('acc')
dev_inter.save(cfg.path_plot_folder + 'interactive_strain_dev.html')
dev_inter

### Lesion time plot deep dive

In [ ]:
time_inter = vis_ns.plot_time_interactive('acc')
time_inter.save(cfg.path_plot_folder + 'interactive_strain_time.html')
time_inter

### Grain plots

In [ ]:
small = vis.plot_dev('acc_small_grain', exp='grain')
large = vis.plot_dev('acc_large_grain', exp='grain')
grain_plot = small | large
grain_plot.save(cfg.path_plot_folder + 'grain.html')
grain_plot

### Imageability effect

In [ ]:
# vis.parse_cond_df(cond_strain='cond_img')
# vis.plot_dev('acc', exp='strain')

### Frequency effect

In [ ]:
# vis.parse_cond_df(cond_strain='cond_wf')
# vis.plot_dev('acc', exp='strain')

### Phonological regularity effect

In [ ]:
# vis.parse_cond_df(cond_strain='cond_pho')
# vis.plot_dev('acc', exp='strain')

### Model weights and biases

In [ ]:
from evaluate import plot_variables
plot_variables(model, cfg.path_plot_folder + 'variables.png')

# Saving results

### Write notebook to html (Must save notebook first)

In [ ]:
# # Only work for manual run
# !jupyter nbconvert --to html --ExecutePreprocessor.store_widget_state=True --output-dir=$cfg.path_model_folder basicOSP_master.ipynba

### Push results to GCP-BQ

In [ ]:
# if cfg.bq_dataset is not None:
#     from meta import write_all_to_bq

#     for attempt in range(10):
#         try:
#             write_all_to_bq(cfg, strain.i_hist, grain.i_hist)
#             print('Results pushed to BQ')
#         except:
#             from time import sleep
#             sleep(10)
#         else:
#             break